In [1]:
import geopandas as gpd
import pandas as pd
import momepy as mm

## generate enclosed tessellation

In [16]:
%%time
blgs = gpd.read_file('data/geometry.gpkg', layer="buildings").to_crs(32648)
streets = gpd.read_file('data/geometry.gpkg', layer="streets").to_crs(32648)

CPU times: total: 8.02 s
Wall time: 8.06 s


In [18]:
# # filter out tunnels
# streets = streets[streets.roadStructure != 'Road In Tunnel']

In [19]:
blgs['uID'] = blgs['uID'].apply(str)
streets['nID'] = streets['nID'].apply(str)

In [10]:
from dask.distributed import Client

In [11]:
import dask.dataframe as dd
from dask.system import cpu_count

In [12]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 383.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64782,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 383.67 GiB
Comm: tcp://127.0.0.1:64841,Total threads: 4
Dashboard: http://127.0.0.1:64850/status,Memory: 47.96 GiB
Nanny: tcp://127.0.0.1:64785,


In [20]:
limit = mm.buffered_limit(blgs, 100)

In [21]:
enclosures = mm.enclosures(streets, limit)

C:\ProgramData\miniconda3\envs\ox\Lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 48N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [23]:
enclosures.to_file('data/geometry_umm.gpkg', layer="enclosures", driver="GPKG")

In [24]:
%time encl_tess = mm.Tessellation(blgs, 'uID', enclosures=enclosures).tessellation

C:\ProgramData\miniconda3\envs\ox\Lib\site-packages\momepy\elements.py:451: FutureWarning: The `query_bulk()` method is deprecated and will be removed in GeoPandas 1.0. You can use the `query()` method instead.
  inp, res = buildings.sindex.query_bulk(
C:\ProgramData\miniconda3\envs\ox\Lib\site-packages\geopandas\array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 48N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


CPU times: total: 29.6 s
Wall time: 1min 25s


In [25]:
encl_tess.to_file('data/geometry_umm.gpkg', layer='encl_tess') #write to disk

In [26]:
client.close()

## tessellation based blocks

In [27]:
blocks = mm.Blocks(encl_tess, streets, blgs, id_name='bID', unique_id='uID')

C:\ProgramData\miniconda3\envs\ox\Lib\site-packages\momepy\elements.py:619: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  weights = libpysal.weights.Queen.from_dataframe(cut, silence_warnings=True)


In [28]:
blocks_gdf = blocks.blocks
blgs['bID'] = blocks.buildings_id
encl_tess['bID'] = blocks.tessellation_id

## link streets

In [29]:
%time blgs['nID'] = mm.get_network_id(blgs, streets, 'nID', min_size=800) 

Snapping:   0%|          | 0/14216 [00:00<?, ?it/s]

CPU times: total: 19min
Wall time: 17min 33s


In [30]:
encl_tess = encl_tess.merge(blgs[['uID', 'nID']], on='uID', how='left')

In [31]:
blocks_gdf.to_file('data/geometry_umm.gpkg', layer='blocks', driver='GPKG')
streets.to_file('data/geometry_umm.gpkg', layer='streets', driver='GPKG')
blgs.to_file('data/geometry_umm.gpkg', layer="buildings", driver="GPKG")
encl_tess.to_file('data/geometry_umm.gpkg', layer="encl_tess", driver="GPKG")